In [1]:
from pyspark.sql.functions import unix_timestamp,col
import numpy as np

In [2]:
#Create dataframe
df = spark.sql("SELECT AAPL_final_csv.date, AAPL_final_csv.close FROM AAPL_final_csv")
df = df.filter(df['Date'] >= "2009-06-22 00:00:00").filter(df['Date'] <= "2018-12-31 00:00:00")
#index
sp500 = spark.sql("SELECT sp500.date, sp500.close FROM sp500").selectExpr("date as date1","close as sp500")
nasdaq100 = spark.sql("SELECT nasdaq100.date, nasdaq100.close FROM nasdaq100").selectExpr("date as date1","close as nasdaq100")
djia = spark.sql("SELECT djia.date, djia.close FROM djia").selectExpr("date as date1","close as DoeJonesIA")
usdcny = spark.sql("SELECT usdcny.date, usdcny.price FROM usdcny").select(unix_timestamp('date', "yyyy/MM/dd") .cast("Timestamp").alias("date1"), col("price").alias("USDtoCNY"))
usdeur = spark.sql("SELECT usdeur.date,usdeur.price FROM usdeur").select(unix_timestamp('date', "yyyy/MM/dd") .cast("Timestamp").alias("date1"), col("price").alias("USDtoEUR"))
usdjpy = spark.sql("SELECT usdjpy.date,usdjpy.price FROM usdjpy").select(unix_timestamp('date', "yyyy/MM/dd") .cast("Timestamp").alias("date1"), col("price").alias("USDtoJPY"))
#related companies stock
anadev = spark.sql("SELECT analogdevice.date, analogdevice.close FROM analogdevice").selectExpr("date as date1","close as AnalogDevice")
jabil = spark.sql("SELECT jabil.date, jabil.close FROM jabil").selectExpr("date as date1","close as Jabil")
microsoft = spark.sql("SELECT microsoft.date, microsoft.close FROM microsoft").selectExpr("date as date1","close as Microsoft")
nidec = spark.sql("SELECT nidec.date, nidec.close FROM nidec").selectExpr("date as date1","close as Nidec")
qualcomm = spark.sql("SELECT qualcomm.date, qualcomm.close FROM qualcomm").selectExpr("date as date1","close as Qualcomm")
tsmc = spark.sql("SELECT tsmc.date, tsmc.close FROM tsmc").selectExpr("date as date1","close as TSMC")

In [3]:
#Join dataframes
df = df.join(sp500,df["Date"]==sp500["Date1"],"Left").drop("Date1")
df = df.join(nasdaq100,df["Date"]==nasdaq100["Date1"],"Left").drop("Date1")
df = df.join(djia,df["Date"]==djia["Date1"],"Left").drop("Date1")
df = df.join(usdcny,df["Date"]==usdcny["Date1"],"Left").drop("Date1")
df = df.join(usdeur,df["Date"]==usdeur["Date1"],"Left").drop("Date1")
df = df.join(usdjpy,df["Date"]==usdjpy["Date1"],"Left").drop("Date1")
df = df.join(anadev,df["Date"]==anadev["Date1"],"Left").drop("Date1")
df = df.join(jabil,df["Date"]==jabil["Date1"],"Left").drop("Date1")
df = df.join(microsoft,df["Date"]==microsoft["Date1"],"Left").drop("Date1")
df = df.join(nidec,df["Date"]==nidec["Date1"],"Left").drop("Date1")
df = df.join(qualcomm,df["Date"]==qualcomm["Date1"],"Left").drop("Date1")
df = df.join(tsmc,df["Date"]==tsmc["Date1"],"Left").drop("Date1").dropna()

In [4]:
#Normalize
rangeMax = np.zeros(len(df.columns) - 2)
rangeMin = np.zeros(len(df.columns) - 2)
for i in range(2,len(df.columns)):
  rangeMax[i-2] = df.select(df.columns[i]).rdd.max()[0]
  rangeMin[i-2] = df.select(df.columns[i]).rdd.min()[0]

In [5]:
df1 = df.select("*")
for i in range(2,len(df.columns)):
  df1 = df1.withColumn("{}_N".format(df.columns[i]), (df1[i]-rangeMin[i-2])/(rangeMax[i-2]-rangeMin[i-2])*100)

for i in range(len(rangeMax)):
  df1 = df1.drop(df1.columns[2])

display(df1)

date,close,sp500_N,nasdaq100_N,DoeJonesIA_N,USDtoCNY_N,USDtoEUR_N,USDtoJPY_N,AnalogDevice_N,Jabil_N,Microsoft_N,Nidec_N,Qualcomm_N,TSMC_N
2009-06-22T00:00:00.000+0000,19.624287,0.6779994849874739,0.3489777709979013,1.0303558464240807,85.00642123287669,20.152217074784932,39.377510040160665,0.3936008225901666,0.0,0.9547328694543463,2.1138670864637588,25.0604416813143,0.0
2009-06-23T00:00:00.000+0000,19.144285,0.7784078454548298,0.31460708407979654,0.944178127177431,84.92080479452052,16.51224354731967,39.17670682730924,0.0,2.052313965887885,1.0190956657563688,2.184329322679216,25.241747083838572,0.4263040673929069
2009-06-24T00:00:00.000+0000,19.46,1.0630622168409927,0.6758964895359538,0.8207975274111998,84.7602739726027,19.027134348113826,39.939759036144565,0.4951752284198897,2.8571429721184294,1.1585496425970918,2.3534386895963184,26.813056543373428,1.22562285489357
2009-06-25T00:00:00.000+0000,19.98,2.0047574648442623,1.1356574372635422,1.744366920691448,84.91010273972603,17.96823295830576,40.06024096385542,1.8410373753437632,2.414487018691629,1.5018257562363073,3.015783710021628,28.9685763289399,1.43877756631211
2009-06-26T00:00:00.000+0000,20.348572,1.9384690681960353,1.205677018698322,1.5623147926197745,84.7923801369863,16.90933156849769,38.91566265060242,1.828340574615046,2.2132797671339937,1.0298229772618992,2.6775649761874285,28.666398643561614,1.3055555369602598
2009-06-29T00:00:00.000+0000,20.28143,2.344487532643684,1.2637069594646353,2.049365778700616,84.7923801369863,16.51224354731967,40.823293172690775,1.7013712976478235,3.420523276479808,1.5769169367750075,2.466178267541052,28.46495021081689,1.2522661896751044
2009-06-30T00:00:00.000+0000,20.347143,1.9589398669318414,1.1585185487255445,1.6084041387139245,84.47131849315066,17.240238252812716,40.9839357429719,1.828340574615046,3.2595574752336995,1.4803700604940988,2.184329322679216,26.67204324480345,0.879251469567353
2009-07-01T00:00:00.000+0000,20.404285,2.154395653567415,1.2239015123072028,1.9138316368828072,84.85659246575345,15.254798146922576,41.54618473895584,1.9299149804447695,4.265593733021879,1.7700085438745226,3.748590966662397,26.59146105139929,0.9591841516340429
2009-07-02T00:00:00.000+0000,20.002857,0.8427475868892581,0.663426783655124,0.7184516841909733,84.56763698630141,18.729318332230307,40.16064257028111,1.7267661687853275,2.3742455683801027,1.0512786730041037,2.9312290265630794,25.624498904603122,0.772675452719124
2009-07-06T00:00:00.000+0000,19.801428,0.9548535327079422,0.5791792714871379,0.9546693731636648,84.88869863013694,18.232958305757787,38.95582329317269,0.9395632539249157,2.0925554161994113,0.8689143774101195,4.298196409142975,26.67204324480345,0.719388783156055


In [6]:
display(df)

date,close,sp500,nasdaq100,DoeJonesIA,USDtoCNY,USDtoEUR,USDtoJPY,AnalogDevice,Jabil,Microsoft,Nidec,Qualcomm,TSMC
2009-06-22T00:00:00.000+0000,19.624287,893.039978,1426.609985,8339.009766,6.8355,0.7214,95.43,23.65,6.61,23.280001,7.53,44.400002,9.034826
2009-06-23T00:00:00.000+0000,19.144285,895.099976,1424.459961,8322.910156,6.8347,0.7104,95.33,23.34,7.12,23.34,7.555,44.490002,9.19403
2009-06-24T00:00:00.000+0000,19.46,900.940002,1447.060059,8299.860352,6.8332,0.718,95.71,23.73,7.32,23.469999,7.615,45.27,9.492537
2009-06-25T00:00:00.000+0000,19.98,920.26001,1475.819946,8472.400391,6.8346,0.7148,95.77,24.790001,7.21,23.790001,7.85,46.34,9.57214
2009-06-26T00:00:00.000+0000,20.348572,918.900024,1480.199951,8438.389648,6.8335,0.7116,95.2,24.780001,7.16,23.35,7.73,46.189999,9.522388
2009-06-29T00:00:00.000+0000,20.28143,927.22998,1483.829956,8529.379883,6.8335,0.7104,96.15,24.68,7.46,23.860001,7.655,46.09,9.502487
2009-06-30T00:00:00.000+0000,20.347143,919.320007,1477.25,8447.0,6.8305,0.7126,96.23,24.780001,7.42,23.77,7.555,45.200001,9.363184
2009-07-01T00:00:00.000+0000,20.404285,923.330017,1481.339966,8504.05957,6.8341,0.7066,96.51,24.860001,7.67,24.040001,8.11,45.16,9.393035
2009-07-02T00:00:00.000+0000,20.002857,896.419983,1446.280029,8280.740234,6.8314,0.7171,95.82,24.700001,7.2,23.370001,7.82,44.68,9.323383
2009-07-06T00:00:00.000+0000,19.801428,898.719971,1441.01001,8324.870117,6.8344,0.7156,95.22,24.08,7.13,23.200001,8.305,45.200001,9.303483
